## Keras ile IMDB dataset
Keras ile Imdb dataset' ini kullanarak yapılan binary classtification(ikili sınıflandırma) örneğidir. Imdb veri seti içinde `50000` adet filmin kritiğinin(sanırım film hakkında yapılan yorumlar) bulunduğu bir dataset. Film değerlendirmelerinden film hakkında olumlu veya olumsuz şekilde bir değerlendirme yapıyor. `Olumlu` ise `1` `olumsuz` ise `0`

Kaynak: Python ile Derin Öğrenme, Frainçois Chollet

Imdb veriseti keras' ta default olarak geliyor yani aslında Imdb veriseti için bir yapı oluşturulmuş yoksa veri hazır gelmiyor.

Veri seti aşağıdaki hücrede indiriliyor.

In [1]:
from keras.datasets import imdb

(train_data, train_labes), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 0s 0us/step


`load_data` Imdb verisetini indirip veriyi `list` dizileri şeklinde döndürüyor.

`num_words` ise listelerin içinde `[0, 100000)` aralığının dışında veri bulunmasını önlüyor.

In [2]:
print("Imdb verisetinin ilk elemanının görünümü:\n", train_data[0])
print("\nİlk filmin kritiği: ", train_labes[0])

Imdb verisetinin ilk elemanının görünümü:
 [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]

İlk filmin kriti

Yukarıdaki hücre çıktısında her sayı bir kelimeyi temsil etmektedir. `Kelime: Sayı` değerlerinin bulunduğu bir JSON dosyası bulunmaktadır.

In [3]:
max([max(sequence) for sequence in train_data])
# Her film kritiğinde [0, 10000) aralığında kelime seçtiğimiz için bu işlemin neticesi 10000' nı geçemez.

9999

In [4]:
word_index = imdb.get_word_index()
print(f"\n\n'Kelime: Sayı' örneği: {list(word_index.keys())[0]}: {list(word_index.values())[0]}")

1646592/1641221 [==============================] - 0s 0us/step


'Kelime: Sayı' örneği: fawn: 34701


Yukarıdaki hücre ile kelimelerin sayı karşılığını tutan `JSON` dosyasını indirip onu bir `dict` haline getirdi: `Kelime: Sayı`

Aşağıdaki hücre ile `Kelime: Sayı` formatındaki `dict`' i `Sayı: Kelime` formatına gönüştürdü.

In [5]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
print(f"'Sayı: Kelime' örneği: {list(reverse_word_index.keys())[0]}: {list(reverse_word_index.values())[0]}")

'Sayı: Kelime' örneği: 34701: fawn


Aşağıdaki hücrede verileri hazırlıyoruz. Hazırlamaktan kasıt list olan verileri Numpy dizilerine çevirmek. Çünkü layer' lara list diziler vermek iyi bir fikir değil onun yerine tensor' lar göndermek iyi fikir.

Elimizdeki bir verileri iki şekilde tensor' a çevirebiliriz.
   1. Bütün verileri eşit boyuta getirip eksik kalan kısımları 0' lar ile doldurmak
   2. (verinin satır sayısı, kelime sayısı) şekilinde bir tensor oluşturup kelimelere karşılık gelen sayıların karışılığını 1 yapmak kalanlarını 0 yapmak. Mesela 560. kelime elimizde olsun. Oluşturduğumuz tensor' un gerekli satırının 560. index değerini 1 yapmak.

`vectorize_sequences` yukarıda bahsedilen ikinci yolu yapmaktadır.

`np.asarray().astype()` ile `label`' ları tutan dizileri numpy dizilerine çeviriyoruz ve `dtype` olarak `int` olan dizileri `float`' a çeviriyoruz.

In [6]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results


x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = np.asanyarray(train_labes).astype('float32')
y_test = np.asanyarray(test_labels).astype('float32')

print(f"types: Data: {type(x_train)}, {type(y_train)}, Label: {type(x_test)}, {type(y_test)}")

types: Data: <class 'numpy.ndarray'>, <class 'numpy.ndarray'>, Label: <class 'numpy.ndarray'>, <class 'numpy.ndarray'>


In [7]:
# sinir ağımızı oluşturturduk
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000, )))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [8]:
# Neural Network Compile
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
)

In [9]:
#Validation veriseti oluşturmak
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [10]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=512,
    validation_data=(x_val, y_val)
)

Epoch 1/20
30/30 [==============================] - 1s 49ms/step - loss: 0.5184 - acc: 0.7783 - val_loss: 0.3916 - val_acc: 0.8645
Epoch 2/20
30/30 [==============================] - 1s 23ms/step - loss: 0.3079 - acc: 0.9013 - val_loss: 0.3190 - val_acc: 0.8761
Epoch 3/20
30/30 [==============================] - 1s 22ms/step - loss: 0.2261 - acc: 0.9260 - val_loss: 0.3063 - val_acc: 0.8756
Epoch 4/20
30/30 [==============================] - 1s 22ms/step - loss: 0.1790 - acc: 0.9442 - val_loss: 0.2779 - val_acc: 0.8878
Epoch 5/20
30/30 [==============================] - 1s 22ms/step - loss: 0.1442 - acc: 0.9550 - val_loss: 0.3072 - val_acc: 0.8774
Epoch 6/20
30/30 [==============================] - 1s 23ms/step - loss: 0.1166 - acc: 0.9659 - val_loss: 0.3184 - val_acc: 0.8788
Epoch 7/20
30/30 [==============================] - 1s 22ms/step - loss: 0.0998 - acc: 0.9702 - val_loss: 0.3036 - val_acc: 0.8847
Epoch 8/20
30/30 [==============================] - 1s 24ms/step - loss: 0.0803 - a

In [11]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])